## 16-02. 도구 바인딩 (Binding Tools)
- LLM 모델이 도구(tool) 를 호출할 수 있으려면 chat 요청을 할 때 모델에 도구 스키마(`tool schema`) 를 전달해야 함
- 도구 호출(tool calling) 기능을 지원하는 LangChain Chat Model 은 `.bind_tools()` 메서드를 구현하여 LangChain 도구 객체, Pydantic 클래스 또는 JSON 스키마 목록을 수신하고 공급자별 예상 형식으로 채팅 모델에 바인딩(binding) 
- 바인딩된 Chat Model 의 후속 호출은 모델 API에 대한 모든 호출에 도구 스키마를 포함

<br>


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

<br>

### LLM에 바인딩할 Tool 정의
- `docstring` 은 가급적 영어로 작성하는 것을 권장

<br>


In [3]:
import re
import requests
from bs4 import BeautifulSoup
from langchain_classic.agents import tool

- `get_word_length` : 단어의 길이를 반환하는 함수

In [4]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

- `add_function` : 두 숫자를 더하는 함수

In [6]:
@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b

- `naver_news_crawl` : 네이버 뉴스 기사를 크롤링하여 본문 내용을 반환하는 함수

In [7]:
@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"

In [8]:
tools = [get_word_length, add_function, naver_news_crawl]

<br>

### `bind_tools()` 로 LLM 에 도구 바인딩
- 질문 분석 및 도구 선택, 인자 추출

In [9]:
from langchain_openai import ChatOpenAI

In [10]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [11]:
llm_with_tools = llm.bind_tools(tools)

<br>

- `name` 은 도구의 이름을 의미
- `args` 는 도구에 전달되는 인자를 의미

In [ ]:
llm_with_tools.invoke("What is the length of the word 'teddynote'?").tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'teddynote'},
  'id': 'call_Ifti1SvGqUzSOzdIP9X8PZ36',
  'type': 'tool_call'}]

<br>

- `llm_with_tools` 와 `JsonOutputToolsParser` 를 연결하여 `tool_calls` 를 parsing 하여 결과를 확인

In [13]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

In [14]:
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)
tool_call_results = chain.invoke("What is the length of the word 'teddynote'?")

In [15]:
print(tool_call_results, end="\n\n==========\n\n")
# 첫 번째 도구 호출 결과
single_result = tool_call_results[0]
# 도구 이름
print(single_result["type"])
# 도구 인자
print(single_result["args"])

[{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}]


get_word_length
{'word': 'teddynote'}


<br>

- **`execute_tool_calls` 함수는 도구를 찾아 `args` 를 전달하여 도구를 실행**
  
  $\rightarrow$ `type` 은 도구의 이름을 의미하고 `args` 는 도구에 전달되는 인자를 의미

In [16]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    
    # 도구 호출 결과 리스트를 순회
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출
        tool_name = tool_call_result["type"]
        tool_args = tool_call_result["args"]

        # 도구 이름과 일치하는 도구를 찾아 실행
        # next() 함수를 사용하여 일치하는 첫 번째 도구
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행
            result = matching_tool.invoke(tool_args)
            print(f"[실행도구] {tool_name}\n[실행결과] {result}")
        else:
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")


In [17]:
execute_tool_calls(tool_call_results)

[실행도구] get_word_length
[실행결과] 9


<br>

### `bind_tools` + `Parser` + `Execution`
- `llm_with_tools` : 도구를 바인딩한 모델
- `JsonOutputToolsParser` : 도구 호출 결과를 파싱하는 파서
- `execute_tool_calls` : 도구 호출 결과를 실행하는 함수

<br>


1. 모델에 도구를 바인딩 
2. 도구 호출 결과를 파싱 
3. 도구 호출 결과를 실행



In [18]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

In [19]:
# bind_tools + Parser + Execution
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [20]:
chain.invoke("What is the length of the word 'teddynote'?")

[실행도구] get_word_length
[실행결과] 9


In [21]:
chain.invoke("114.5 + 121.2")
print(114.5 + 121.2)

[실행도구] add_function
[실행결과] 235.7
235.7


In [23]:
# chain.invoke(
#     "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
# )

<br>

### `bind_tools` > Agent & `AgentExecutor` 로 대체
* `bind_tools()` 는 모델에 사용할 수 있는 스키마(도구)를 제공
* **`AgentExecutor` 는 실제로 llm 호출, 올바른 도구로 라우팅, 실행, 모델 재호출 등을 위한 실행 루프를 생성**

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

from langchain_classic.agents import create_tool_calling_agent
from langchain_classic.agents import AgentExecutor

- Agent프롬프트 생성

In [30]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [31]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

- 정의한 도구 사용

In [32]:
tools = [get_word_length, add_function, naver_news_crawl]

- Agent 생성

In [33]:
agent = create_tool_calling_agent(llm, tools, prompt)

- `AgentExecutor` 생성

In [34]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

- Agent 실행

In [39]:
result = agent_executor.invoke({"input": "How many letters in the word `teddynote`?"})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'teddynote'}`


9The word "teddynote" has 9 letters.

> Finished chain.
The word "teddynote" has 9 letters.


In [38]:
result = agent_executor.invoke(
    {"input": "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"}
)

print(result["output"])
print("==========\n")
print(114.5 + 121.2 + 34.2 + 110.1)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 235.7, 'b': 34.2}`


269.9
Invoking: `add_function` with `{'a': 269.9, 'b': 110.1}`


380.0114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

> Finished chain.
114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

380.0


<br>

<hr>

<br>

## 16-03. 에이전트 (Agent)

<br>

### 도구 호출 에이전트(Tool Calling Agent)
- **도구 호출을 사용하면 모델이 하나 이상의 도구(tool) 가 호출되어야 하는 시기를 감지하고 해당 도구에 전달해야 하는 입력 으로 전달 가능**
- **API 호출에서 도구를 설명하고 모델이 이러한 도구를 호출하기 위한 인수가 포함된 JSON과 같은 구조화된 객체를 출력하도록 지능적으로 선택**
- **도구 API 의 목표는 일반 텍스트 완성이나 채팅 API를 사용하여 수행할 수 있는 것보다 더 안정적으로 유효하고 유용한 도구 호출(tool call) 을 반환하는 것**
  
  $\rightarrow$ **구조화된 출력을 도구 호출 채팅 모델에 여러 도구를 바인딩하고, 모델이 호출할 도구를 선택할 수 있다는 사실과 결합하여 쿼리가 해결될 때까지 반복적으로 도구를 호출하고 결과를 수신하는 에이전트 생성 가능**

<br>



In [42]:
from langchain.tools import tool
from typing import List, Dict, Annotated
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

- 도구 생성

In [43]:
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [44]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [45]:
print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")
print(f"도구 이름: {python_repl_tool.name}")
print(f"도구 설명: {python_repl_tool.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword
도구 이름: python_repl_tool
도구 설명: Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.


<br>

### Agent 프롬프트 생성
- `chat_history` : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능)
- `agent_scratchpad` : 에이전트가 임시로 저장하는 변수
- `input` : 사용자의 입력

In [46]:
from langchain_core.prompts import ChatPromptTemplate

In [47]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

<br>

### `AgentExecutor`
- **`AgentExecutor`는 도구를 사용하는 에이전트를 실행하는 클래스**

<br>

#### 주요 속성
- `agent`: 실행 루프의 각 단계에서 계획을 생성하고 행동을 결정하는 에이전트
- `tools`: 에이전트가 사용할 수 있는 유효한 도구 목록
- `return_intermediate_steps`: 최종 출력과 함께 에이전트의 중간 단계 경로를 반환할지 여부
- `max_iterations`: 실행 루프를 종료하기 전 최대 단계 수
- `max_execution_time`: 실행 루프에 소요될 수 있는 최대 시간
- `early_stopping_method`: 에이전트가 `AgentFinish`를 반환하지 않을 때 사용할 조기 종료 방법. (`"force"` or `"generate"`)
  - `"force"` : 시간 또는 반복 제한에 도달하여 중지되었다는 문자열을 반환합니다.
  - `"generate"` : 에이전트의 LLM 체인을 마지막으로 한 번 호출하여 이전 단계에 따라 최종 답변을 생성합니다.
- `handle_parsing_errors`: 에이전트의 출력 파서에서 발생한 오류 처리 방법. (`True`, `False`, 또는 오류 처리 함수)
- `trim_intermediate_steps`: 중간 단계를 트리밍하는 방법. (-1 trim 하지 않음, 또는 트리밍 함수)

<br>

#### 최적화 요령
- `max_iterations`와 `max_execution_time`을 적절히 설정하여 실행 시간 관리
- `trim_intermediate_steps`를 활용하여 메모리 사용량 최적화
- 복잡한 작업의 경우 `stream` 메서드를 사용하여 단계별 결과 모니터링

In [48]:
from langchain_classic.agents import AgentExecutor

In [49]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    max_execution_time=10,
    handle_parsing_errors=True,
)

In [53]:
result = agent_executor.invoke({"input": "iris 데이터셋의 첫 5행을 볼 수 있는 코드를 작성해주세요"})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...
Iris 데이터셋의 첫 5행을 출력하는 코드는 다음과 같습니다. 이 코드는 Python의 pandas 라이브러리를 사용하여 작성되었습니다.

```python
import pandas as pd
from sklearn.datasets import load_iris

# Iris 데이터셋 로드
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# 첫 5행 출력
print(iris_df.head())
```

이 코드를 실행하면 Iris 데이터셋의 첫 5행이 출력됩니다.

> Finished chain.
Agent 실행 결과:
Iris 데이터셋의 첫 5행을 출력하는 코드는 다음과 같습니다. 이 코드는 Python의 pandas 라이브러리를 사용하여 작성되었습니다.

```python
import pandas as pd
from sklearn.datasets import load_iris

# Iris 데이터셋 로드
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# 첫 5행 출력
print(iris_df.head())
```

이 코드를 실행하면 Iris 데이터셋의 첫 5행이 출력됩니다.


<br>

### Stream 출력으로 단계별 결과 확인
- **`stream()`의 출력은 (Action, Observation) 쌍 사이에서 번갈아 나타나며, 최종적으로 에이전트가 목표를 달성했다면 답변으로 마무리**

<table>
<thead>
<tr>
<th>출력</th>
<th>내용</th>
</tr>
</thead>
<tbody>
<tr>
<td>Action</td>
<td><code>actions</code>: AgentAction 또는 그 하위 클래스<br><code>messages</code>: 액션 호출에 해당하는 채팅 메시지</td>
</tr>
<tr>
<td>Observation</td>
<td><code>steps</code>: 현재 액션과 그 관찰을 포함한 에이전트가 지금까지 수행한 작업의 기록<br><code>messages</code>: 함수 호출 결과(즉, 관찰)를 포함한 채팅 메시지</td>
</tr>
<tr>
<td>Final Answer</td>
<td><code>output</code>: AgentFinish<br><code>messages</code>: 최종 출력을 포함한 채팅 메시지</td>
</tr>
</tbody>
</table>

In [54]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)

In [ ]:
# 스트리밍 모드 실행
result = agent_executor.stream({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."}) 

for step in result:
    # 중간 단계 출력
    print(step)

<br>

### 이전 대화내용 기억하는 Agent
- **이전의 대화내용을 기억하기 위해서는 `RunnableWithMessageHistory` 를 사용하여 `AgentExecutor` 를 감싸줌**

In [58]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [73]:
store = {}

- `session_id` 를 기반으로 세션 기록을 가져오는 함수

In [74]:
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

- 채팅 메시지 기록이 추가된 에이전트를 생성

In [75]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

- 질의에 대한 답변을 스트리밍으로 출력 요청

In [76]:
response = agent_with_chat_history.stream(
    {"input": "안녕? 내 이름은 테디야!"},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    print(step)

{'output': '안녕하세요, 테디! 만나서 반가워요. 어떻게 도와드릴까요?', 'messages': [AIMessage(content='안녕하세요, 테디! 만나서 반가워요. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]}


In [77]:
response = agent_with_chat_history.stream(
    {
        "input": "내 이메일 주소는 teddy@teddynote.com 이야. 회사 이름은 테디노트 주식회사야."
    },
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    print(step)

{'output': '안녕하세요, 테디노트 주식회사에 대해 어떻게 도와드릴까요? 이메일 주소와 관련된 질문이나 요청이 있으신가요?', 'messages': [AIMessage(content='안녕하세요, 테디노트 주식회사에 대해 어떻게 도와드릴까요? 이메일 주소와 관련된 질문이나 요청이 있으신가요?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]}


<br>

<hr>

<br>

## 16-05. Iteration 기능과 사람 개입 (Human-in-the-loop)

<br>

